In [6]:
from astropy.io import fits
import aplpy
import matplotlib.pyplot as plt
from astropy.wcs import WCS
import numpy as np

def crop_3D_fits(input_name, output_name, x, y,
                crop_radius_pix):
    '''Take a 3D spectral cube style FITS file and 
    crop around a given pixel (x,y) with a radius of 
    crop_radius_pix in pixel coords'''
    with fits.open(input_name) as hdu:
        data = hdu[1].data
        header = hdu[1].header
        current_wcs = WCS(header)
        print('Current data shape',data.shape)

        ##Find the current central pixel
        current_central_pix1 = int(x)
        current_central_pix2 = int(y)
        
        ##Find the RA/Dec of that pixel
        cent_pix_ra, cent_pix_dec, _ = current_wcs.all_pix2world(current_central_pix1,
                                                                    current_central_pix2, 0, 0)
        print("Current central pixels",current_central_pix1,current_central_pix2)
        print("RA, Dec values",cent_pix_ra, cent_pix_dec)
        
        ##Setup slicing levels
        ##Remember slicing goes up to one below so stick
        ##a + 1 on the upper values to be even about central pix
        low1 = current_central_pix1 - crop_radius_pix
        up1 = current_central_pix1 + crop_radius_pix + 1
        
        low2 = current_central_pix2 - crop_radius_pix
        up2 = current_central_pix2 + crop_radius_pix + 1
        
        ##Make the cropped slice
        cropped_data = data[:,low2:up2,low1:up1]
        
        ##Find the central pixel of the slice
        new_central_pix1 = int(np.floor(cropped_data.shape[2]/2))
        new_central_pix2 = int(np.floor(cropped_data.shape[1]/2))
        
        print("New data shape",cropped_data.shape)
        print("New central pixels",new_central_pix1,new_central_pix2)
        
        ##Now we have to modify the current contents of the FITS
        
        ##This shows we have changed the length of these two axes
        hdu[1].header['NAXIS1'] = cropped_data.shape[2]
        hdu[1].header['NAXIS2'] = cropped_data.shape[1]
        
        ##This means we have changed the central pixel
        ##FITS files are 1 indexed, NOT 0 indexed, so
        ##need to add 1
        hdu[1].header['CRPIX1'] = new_central_pix1
        hdu[1].header['CRPIX2'] = new_central_pix2
        
        ##Have to make sure the RA/Dec value of the
        ##new central pixel is correct
        hdu[1].header['CRVAL1'] = float(cent_pix_ra)
        hdu[1].header['CRVAL2'] = float(cent_pix_dec)
        
        ##Replace the data
        hdu[1].data = cropped_data
        ##write out the data to a new filename
        hdu.writeto(output_name, overwrite=True)

xpix = 26
ypix = 28
radius = 5
        
##Do the cropping
crop_3D_fits('1009_629_2006A_individual_cubes_3D.fits',
             'cropped_1009_629_2006A_individual_cubes_3D.fits',
             xpix, ypix, radius)


Current data shape (6252, 50, 39)
Current central pixels 26 28
RA, Dec values 59.55978342781352 -2.7468012235896784
New data shape (6252, 11, 11)
New central pixels 5 5
